In [ ]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import os
pylab.rcParams['figure.figsize'] = (25, 25)


In [ ]:
dataDir='../../data/Annotationsdaten/all annotated'
annFile= os.path.join(dataDir,"all_coco.json")
bugs = COCO(annFile)


In [ ]:
cats = bugs.loadCats(bugs.getCatIds())
for i in bugs.imgs:
    annIds = bugs.getAnnIds(imgIds=i, catIds=range(4), iscrowd=None)
    anns = bugs.loadAnns(annIds)
    print(len (anns))
    img = io.imread(os.path.join(dataDir,bugs.imgs[i]["file_name"]))
    plt.axis('off')
    plt.imshow(img)
    bugs.showAnns(anns)
    plt.show()

In [ ]:
img = io.imread("../../data/test_segmentation/org_image_comb_big_stich.tif")
plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
mask = io.imread("../../data/test_segmentation/mask.tif")
plt.imshow(mask, cmap='gray')
plt.show()

In [ ]:
from skimage import measure
from shapely.geometry import Polygon, MultiPolygon

In [ ]:
contours = measure.find_contours(mask, 0.5, positive_orientation='low')
# Display the image and plot all contours found
fig, ax = plt.subplots()
ax.imshow(mask, cmap=plt.cm.gray)

for contour in contours:
    ax.plot(contour[:, 1], contour[:, 0], linewidth=2)

ax.axis('image')
ax.set_xticks([])
ax.set_yticks([])
plt.show()

In [ ]:
segmentations = []
polygons = []
is_crowd = 0
image_id = 5
category_id = 3
annotation_id = 500
a = []
min_area = 500
for contour in contours:
    # Make a polygon and simplify it
    poly = Polygon(contour)
    if poly.area < min_area:
        continue
    poly = poly.simplify(1.0, preserve_topology=False)
    segmentation = np.array(poly.exterior.coords).ravel().tolist()
    d = {
        "id": annotation_id,
        "image_id": image_id,
        "category_id": category_id,
        "iscrowd": is_crowd,
        "area": poly.area,
        "bbox": [round(x) for x in poly.bounds],
        "segmentation": segmentation,
        }
    a.append(d)
    annotation_id += 1

In [ ]:
s = {
    "info": {
        "date_created": "2021-03-28 22:10:50.716335"
    },
    "licenses": [
        {
            "id": 1,
            "name": "Attribution-NonCommercial-ShareAlike License",
            "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/"
        }
    ],
    "categories": [
        {
            "id": 1,
            "name": "Mesembryhmus purpuralis",
            "supercategory": "None"
        },
        {
            "id": 2,
            "name": "Smerinthus ocellata",
            "supercategory": "None"
        },
        {
            "id": 3,
            "name": "Mesembryhmus purpuralis",
            "supercategory": "None"
        },
        {
            "id": 4,
            "name": "bug",
            "supercategory": "None"
        }
    ],
    "images": [
        {
            "id": 0,
            "file_name": "Smerinthus_ocellata.jpg",
            "width": 3888,
            "height": 5184,
            "date_captured": "2021-03-28 22:10:50.716292",
            "license": 1,
            "coco_url": "",
            "flickr_url": ""
        },
        {
            "id": 1,
            "file_name": "DSC_6754.JPG",
            "width": 3264,
            "height": 4928,
            "date_captured": "2021-03-28 22:10:50.716292",
            "license": 1,
            "coco_url": "",
            "flickr_url": ""
        },
        {
            "id": 2,
            "file_name": "Mesembryhnus_purpuralis.JPG",
            "width": 3264,
            "height": 4928,
            "date_captured": "2021-03-28 22:10:50.716292",
            "license": 1,
            "coco_url": "",
            "flickr_url": ""
        },
        {
            "id": 3,
            "file_name": "bug.tif",
            "width": 2912,
            "height": 1440,
            "date_captured": "at some timepoint",
            "license": 1,
            "coco_url": "",
            "flickr_url": ""
        }
    ],
    "annotations": a,
}


In [ ]:
import json
y = json.dumps(s)
print(y)
with open('../../data/anno.json', 'w') as f:
    f.write(y)

In [ ]:
bugs = COCO('../../data/anno.json')

In [ ]:
annIds = bugs.getAnnIds(imgIds=[5], catIds=range(5), iscrowd=None)
anns = bugs.loadAnns(annIds)
print(len (anns))
plt.axis('off')
plt.imshow(img, cmap="gray")
bugs.showAnns(a)
plt.show()

In [ ]:
img.shape